In [151]:
import numpy as np
import pandas as pd
import datetime

In [152]:
user_df = pd.read_csv('User.csv')


In [153]:
symptom_df = pd.read_csv('Symptom.csv')

In [154]:
period_df = pd.read_csv('periods_cleaned.csv')

In [155]:
period_df = period_df.rename(index=str, columns={"start_date": "period_start", "end_date": "period_end"})

In [156]:
print(user_df.head())

    id  dob  cycle_length_initial  period_length_initial
0  252  NaN                    28                      6
1  253  NaN                    30                      5
2  254  NaN                    28                      6
3  255  NaN                    30                      4
4  256  NaN                    25                      6


In [157]:
print(symptom_df.shape, period_df.shape)

(13512, 13) (34934, 10)


In [158]:
# user_df = user_df.rename(index=str, columns={"id":"user_id"})
# period_df = period_df.rename(index=str, columns={"User_id":"user_id"})

In [159]:
symptoms = symptom_df.merge(period_df, how='left', on='user_id')

In [160]:
symptoms[symptoms.user_id == 252]

         

,id_x,user_id,acne,backache,bloating,cramp,diarrhea,dizzy,headache,mood,...,date,Unnamed: 0,dob,cycle_length_initial,period_length_initial,id_y,period_start_cycle_start,period_end,prev_cycle_end_date,cycle_end_date
0,127,252,0,0,0,0,0,0,0,0,...,11/4/15,0.0,NaN,28 days 00:00:00.000000000,6 days 00:00:00.000000000,352.0,2015-04-05,2015-04-10,2015-04-04,2015-05-03
75,142,252,50,50,0,0,0,0,0,0,...,13/4/15,0.0,NaN,28 days 00:00:00.000000000,6 days 00:00:00.000000000,352.0,2015-04-05,2015-04-10,2015-04-04,2015-05-03
203,171,252,0,0,0,0,0,0,0,0,...,17/4/15,0.0,NaN,28 days 00:00:00.000000000,6 days 00:00:00.000000000,352.0,2015-04-05,2015-04-10,2015-04-04,2015-05-03


In [161]:
# df_new['period_start'] = pd.to_datetime(df_new['period_start'], errors='coerce', infer_datetime_format=True)

In [162]:
# null_columns=df_new.columns[df_new.isnull().any()]

In [163]:
# symptoms.dropna(inplace=True)

In [164]:
symptoms.drop(['id_x', 'Unnamed: 0', 'id_y', 'dob'], axis=1, inplace=True)

In [165]:
for label in ['period_start_cycle_start', 'period_end', 'prev_cycle_end_date', 'cycle_end_date']:
    symptoms[label] = pd.to_datetime(symptoms[label])

In [166]:
symptoms['date'] = pd.to_datetime(symptoms['date'], errors='coerce', dayfirst=True)

In [167]:
symptoms['cycle_length_initial'] = pd.to_timedelta(symptoms['cycle_length_initial'], unit="D")
symptoms['period_length_initial'] = pd.to_timedelta(symptoms['period_length_initial'], unit="D")

In [168]:
symptoms['prev_cycle_start_date'] = symptoms['prev_cycle_end_date'] - symptoms['cycle_length_initial']

In [169]:
symptoms['cycle_percentage'] = ( symptoms['date'] - symptoms['period_start_cycle_start'] ) / symptoms['cycle_length_initial']

In [170]:
within_cycle =  symptoms[(symptoms['cycle_percentage'] > 0) & (symptoms['cycle_percentage'] < 1)]

In [171]:
within_last_cycle = symptoms[(symptoms['cycle_percentage'] > -1) & (symptoms['cycle_percentage'] < 0)]

In [172]:
within_last_cycle['cycle_percentage'] = ( within_last_cycle['date'] - within_last_cycle['prev_cycle_start_date'] ) / within_last_cycle['cycle_length_initial']

/home/deepan/Dropbox/Projects/tender_elion/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [173]:
print(within_cycle.shape, within_last_cycle.shape)

(10398, 20) (6519, 20)


In [174]:
final_df = pd.concat([within_cycle, within_last_cycle], axis=0)

In [175]:
final_df.shape

(16917, 20)

In [179]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16917 entries, 0 to 195054
Data columns (total 20 columns):
user_id                     16917 non-null int64
acne                        16917 non-null int64
backache                    16917 non-null int64
bloating                    16917 non-null int64
cramp                       16917 non-null int64
diarrhea                    16917 non-null int64
dizzy                       16917 non-null int64
headache                    16917 non-null int64
mood                        16917 non-null int64
nausea                      16917 non-null int64
sore                        16917 non-null int64
date                        16917 non-null datetime64[ns]
cycle_length_initial        16917 non-null timedelta64[ns]
period_length_initial       16917 non-null timedelta64[ns]
period_start_cycle_start    16917 non-null datetime64[ns]
period_end                  16917 non-null datetime64[ns]
prev_cycle_end_date         16917 non-null datetime64[ns]
c

In [180]:
final_df.describe()

,user_id,acne,backache,bloating,cramp,diarrhea,dizzy,headache,mood,nausea,sore,cycle_length_initial,period_length_initial,cycle_percentage
count,16917.000000,16917.000000,16917.000000,16917.000000,16917.000000,16917.000000,16917.000000,16917.000000,16917.000000,16917.000000,16917.000000,16917,16917,16917.000000
mean,3522.971567,10.804753,17.293255,18.469764,24.758704,6.510315,5.883667,14.976651,19.936218,6.607200,14.651830,28 days 15:55:08.884553,5 days 14:46:22.195424,0.430031
std,1869.271500,21.203897,28.427643,28.945317,32.859893,18.070838,17.095204,26.790871,31.260061,18.696963,27.330372,2 days 17:08:38.143691,2 days 03:09:37.301652,0.353506
min,252.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25 days 00:00:00,2 days 00:00:00,0.025000
25%,2025.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,28 days 00:00:00,5 days 00:00:00,0.103448
50%,3375.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,28 days 00:00:00,5 days 00:00:00,0.285714
75%,5045.000000,14.000000,30.000000,33.000000,46.000000,0.000000,0.000000,23.000000,35.000000,0.000000,20.000000,29 days 00:00:00,7 days 00:00:00,0.821429
max,7054.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,40 days 00:00:00,35 days 00:00:00,1.000000


In [181]:
final_df.to_pickle('processed_symptoms.pkl')

In [182]:
# for x in range(25, 40):
#     print(x, len(final_df[final_df.cycle_length_initial == x]))

25 0
26 0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
38 0
39 0
